In [14]:
from datetime import date, timedelta
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima_model import ARIMA
import matplotlib as plp
import lightgbm as lgb

In [15]:
df_train = pd.read_csv(
    '../Data/train_set.csv', usecols=[1, 2, 3, 4],
    converters={'unit_sales': lambda u: np.log1p(
        float(u)) if float(u) > 0 else 0},
    parse_dates=["date"]
)

In [16]:
df_test = pd.read_csv(
    "../Data/test.csv", usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
).set_index(
    ['store_nbr', 'item_nbr', 'date']
)

In [17]:
items = pd.read_csv(
    "../Data/items.csv",
).set_index("item_nbr")

In [18]:
df_train = df_train.set_index(
    ["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(
        level=-1).fillna(0)
df_train.columns = df_train.columns.get_level_values(1)

In [19]:
items = items.reindex(df_train.index.get_level_values(1))

In [20]:
def get_timespan(df, dt, minus, periods):
    return df[
        pd.date_range(dt - timedelta(days=minus), periods=periods)
    ]

In [21]:
def prepare_dataset(t2017, is_train=True):
    X = pd.DataFrame({
        "mean_3_2017": get_timespan(df_train, t2017, 3, 3).mean(axis=1).values,
        "mean_7_2017": get_timespan(df_train, t2017, 7, 7).mean(axis=1).values,
        "mean_14_2017": get_timespan(df_train, t2017, 14, 14).mean(axis=1).values,
        "mean_16_2017": get_timespan(df_train, t2017, 16, 16).mean(axis=1).values
    })
    if is_train:
        y = df_train[
            pd.date_range(t2017, periods=16)
        ].values
        return X, y
    return X

In [22]:
print("Preparing dataset...")
t2017 = date(2017, 5, 16)
X_l, y_l = [], []
for i in range(9):
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = prepare_dataset(t2017 + delta)
    X_l.append(X_tmp)
    y_l.append(y_tmp)
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)
del X_l, y_l

Preparing dataset...


In [23]:
X_val, y_val = prepare_dataset(date(2017, 7, 23))

In [24]:
X_test = prepare_dataset(date(2017, 8, 16), is_train=False)

In [25]:
print("Training and predicting models...")
params = {
    'num_leaves': 80,
    'objective': 'regression',
    'min_data_in_leaf': 200,
    'learning_rate': 0.02,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.7,
    'bagging_freq': 1,
    'metric': 'l2',
    'num_threads': 16
}

MAX_ROUNDS = 5000
val_pred = []
test_pred = []
cate_vars = []
for i in range(16):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    dtrain = lgb.Dataset(
        X_train, label=y_train[:, i],
        categorical_feature=cate_vars,
    )
    
    dval = lgb.Dataset(
        X_val, label=y_val[:, i], reference=dtrain,
        categorical_feature=cate_vars)
    
    bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS,
        valid_sets=[dtrain, dval], early_stopping_rounds=50, verbose_eval=50
    )
    
    print("\n".join(("%s: %.2f" % x) for x in sorted(
        zip(X_train.columns, bst.feature_importance("gain")),
        key=lambda x: x[1], reverse=True
    )))
    val_pred.append(bst.predict(
        X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))
    test_pred.append(bst.predict(
        X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))

print("Validation mse:", mean_squared_error(
    y_val, np.array(val_pred).transpose()))



Training and predicting models...
Step 1


/Users/arkarmin/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1186: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 50 rounds.
[50]	training's l2: 0.45991	valid_1's l2: 0.577641
[100]	training's l2: 0.376471	valid_1's l2: 0.459938
[150]	training's l2: 0.364744	valid_1's l2: 0.43592
[200]	training's l2: 0.362864	valid_1's l2: 0.429516
[250]	training's l2: 0.36235	valid_1's l2: 0.42762
[300]	training's l2: 0.362052	valid_1's l2: 0.426967
[350]	training's l2: 0.361826	valid_1's l2: 0.426736
[400]	training's l2: 0.361616	valid_1's l2: 0.426613
[450]	training's l2: 0.361423	valid_1's l2: 0.426609
[500]	training's l2: 0.361236	valid_1's l2: 0.426563
[550]	training's l2: 0.361051	valid_1's l2: 0.426514
Early stopping, best iteration is:
[544]	training's l2: 0.361074	valid_1's l2: 0.426506
mean_7_2017: 7853531.27
mean_14_2017: 7769878.44
mean_16_2017: 1755879.00
mean_3_2017: 1138635.64
Step 2


/Users/arkarmin/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1186: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 50 rounds.
[50]	training's l2: 0.473812	valid_1's l2: 0.481605
[100]	training's l2: 0.388944	valid_1's l2: 0.393762
[150]	training's l2: 0.377053	valid_1's l2: 0.380619
[200]	training's l2: 0.375119	valid_1's l2: 0.378371
[250]	training's l2: 0.374575	valid_1's l2: 0.377851
[300]	training's l2: 0.374244	valid_1's l2: 0.377687
[350]	training's l2: 0.373999	valid_1's l2: 0.377623
[400]	training's l2: 0.373772	valid_1's l2: 0.377551
[450]	training's l2: 0.373567	valid_1's l2: 0.377535
[500]	training's l2: 0.373367	valid_1's l2: 0.377518
[550]	training's l2: 0.373182	valid_1's l2: 0.377526
Early stopping, best iteration is:
[518]	training's l2: 0.373298	valid_1's l2: 0.37751
mean_14_2017: 11673125.24
mean_7_2017: 5071917.07
mean_16_2017: 1957731.69
mean_3_2017: 191834.47
Step 3


...

Training until validation scores don't improve for 50 rounds.
[50]	training's l2: 0.539635	valid_1's l2: 0.612142
[100]	training's l2: 0.461833	valid_1's l2: 0.51955
[150]	training's l2: 0.450751	valid_1's l2: 0.502711
[200]	training's l2: 0.448773	valid_1's l2: 0.498588
[250]	training's l2: 0.448164	valid_1's l2: 0.497312
[300]	training's l2: 0.44781	valid_1's l2: 0.496861
[350]	training's l2: 0.447512	valid_1's l2: 0.496634
[400]	training's l2: 0.447245	valid_1's l2: 0.496537
[450]	training's l2: 0.446985	valid_1's l2: 0.496402
[500]	training's l2: 0.446744	valid_1's l2: 0.496341
[550]	training's l2: 0.446513	valid_1's l2: 0.49627
[600]	training's l2: 0.446299	valid_1's l2: 0.496227
[650]	training's l2: 0.446089	valid_1's l2: 0.496173
[700]	training's l2: 0.445889	valid_1's l2: 0.496149
[750]	training's l2: 0.445698	valid_1's l2: 0.496126
Early stopping, best iteration is:
[740]	training's l2: 0.445738	valid_1's l2: 0.496105
mean_16_2017: 14118852.01
mean_14_2017: 2759440.48
mean_7_2

/Users/arkarmin/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1186: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 50 rounds.
[50]	training's l2: 0.532407	valid_1's l2: 0.700643
[100]	training's l2: 0.460248	valid_1's l2: 0.597684
[150]	training's l2: 0.450073	valid_1's l2: 0.576263
[200]	training's l2: 0.448307	valid_1's l2: 0.570487
[250]	training's l2: 0.447751	valid_1's l2: 0.568552
[300]	training's l2: 0.447413	valid_1's l2: 0.567805
[350]	training's l2: 0.447138	valid_1's l2: 0.567502
[400]	training's l2: 0.446886	valid_1's l2: 0.567286
[450]	training's l2: 0.446642	valid_1's l2: 0.567162
[500]	training's l2: 0.446414	valid_1's l2: 0.56705
[550]	training's l2: 0.44619	valid_1's l2: 0.566965
[600]	training's l2: 0.445967	valid_1's l2: 0.566895
[650]	training's l2: 0.445753	valid_1's l2: 0.566813
[700]	training's l2: 0.445553	valid_1's l2: 0.566739
[750]	training's l2: 0.445356	valid_1's l2: 0.566759
Early stopping, best iteration is:
[707]	training's l2: 0.445526	valid_1's l2: 0.566726
mean_16_2017: 13096858.84
mean_14_2017: 2554206.61
mean_7_

/Users/arkarmin/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1186: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 50 rounds.
[50]	training's l2: 0.539131	valid_1's l2: 0.552223
[100]	training's l2: 0.464018	valid_1's l2: 0.475322
[150]	training's l2: 0.453404	valid_1's l2: 0.464057
[200]	training's l2: 0.451575	valid_1's l2: 0.462185
[250]	training's l2: 0.451001	valid_1's l2: 0.461728
[300]	training's l2: 0.450644	valid_1's l2: 0.461588
[350]	training's l2: 0.450362	valid_1's l2: 0.461529
[400]	training's l2: 0.450099	valid_1's l2: 0.461486
[450]	training's l2: 0.449853	valid_1's l2: 0.461442
[500]	training's l2: 0.449614	valid_1's l2: 0.46142
[550]	training's l2: 0.449384	valid_1's l2: 0.461414
[600]	training's l2: 0.449165	valid_1's l2: 0.461376
[650]	training's l2: 0.44896	valid_1's l2: 0.46133
[700]	training's l2: 0.448749	valid_1's l2: 0.461322
[750]	training's l2: 0.448549	valid_1's l2: 0.461329
Early stopping, best iteration is:
[701]	training's l2: 0.448745	valid_1's l2: 0.46132
mean_16_2017: 13547175.80
mean_14_2017: 2891551.01
mean_7_20

In [26]:
print("Making submission...")
y_test = np.array(test_pred).transpose()
df_preds = pd.DataFrame(
    y_test, index=df_train.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

submission = df_test[["id"]].join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)
submission.to_csv('modified_1.csv', float_format='%.4f', index=None)

Making submission...
